# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
import random
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer
import statsmodels.api as sm

from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

# %autosave 50

## Get user comments

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1        
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [ ]:
# comments_df = getDF(data_path + file_name, 200000)
# comments_df.loc[0,'reviewText']
# print(comments_df.shape)
# comments_df.head(1)

In [4]:
# Reads records
pickle_in = open(data_path + "amzn_200k.pickle","rb")
comments_df = pickle.load(pickle_in)

## Subsample

In [5]:
# Subsampling from Amazon reviews
NB_SAMPLES = 5000  # up to 200k, then change the input file

In [7]:
reviews = comments_df.loc[:NB_SAMPLES, :]
print(reviews.shape)
reviews.head(3)

(5001, 9)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3R5OBKS7OM2IR,0000143502,Rebecca L. Johnson,"[0, 0]",This has some great tips as always and is help...,5.0,Alton... nough said,1358380800,"01 17, 2013"
1,A3R5OBKS7OM2IR,0000143529,Rebecca L. Johnson,"[0, 0]",This is a great pastry guide. I love how Alto...,5.0,Ah Alton...,1380672000,"10 2, 2013"
2,AH3QC2PC1VTGP,0000143561,Great Home Cook,"[2, 4]",I have to admit that I am a fan of Giada's coo...,2.0,Don't waste your money,1216252800,"07 17, 2008"


In [8]:
# comments_df.iloc[random.sample(range(25000), 2), :]

### Load and apply TFIDF 

In [9]:
MAX_FEATURES = 2000

In [15]:
if True:
    # Get pre-fit TFIDF
    pickle_in = open("pickles/tfidf_25kBalancedSamples_20kFeats.pkl","rb")
    tfidf = pickle.load(pickle_in)
    pickle_in.close()
    Xa_test = tfidf.transform(reviews['reviewText'])
else:
    # Fit TFIDF on test, with pre-trained vocabulary
    pickle_in = open("pickles/tfidf_vocab_25kBalancedSamples_20kFeats.pkl","rb")
    vocab = pickle.load(pickle_in)
    pickle_in.close()
    tfidf = TfidfVectorizer(lowercase=True, 
                        stop_words='english', 
                        max_features=MAX_FEATURES,
                        vocabulary=vocab,
                        norm='l2',            # normalize each review
                        use_idf=True)        # Keep high weight for most common words
    Xa_test = tfidf.fit_transform(reviews['reviewText'])
ya_test = reviews['overall']

In [16]:
print(Xa_test.shape)
print(ya_test.shape)

(5001, 20000)
(5001,)


### Gradient Boosting Classifier

In [12]:
# Gradient Boosting Classifier parameters
N_TREES = 300 # math.floor(np.sqrt(NB_SAMPLES) * 1.2)
LEARN_RATE = 0.1
MAX_DEPTH = 15
MIN_IN_LEAF = 2 #7

In [ ]:
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                max_depth=MAX_DEPTH,
                                random_state=0)

In [13]:
if False:
    gb_pipe = Pipeline([('vect', tfidf), ('gb', gbc)])
    gb_pipe.fit(X_train, y_train)
    pickle.dump(gb_pipe, open('pickles/GBC_balanced_comments_'
                           + str(N_TREES) + '_trees_' 
                           + str(LEARN_RATE) + '_lr_' 
                           + str(MAX_DEPTH) + '_maxdpth_'
                           + str(MIN_IN_LEAF) + '_minleaf_'
                           + str(MAX_FEATURES) + '_feats_'
                           + '.pkl', 'wb'))
else:
    pickle_in = open("pickles/GBC_balanced_comments_300_trees_0.1_lr_15_maxdpth_2_minleaf_20000_feats_.pkl",
                     "rb")
    gb_pipe = pickle.load(pickle_in)
    pickle_in.close()

In [17]:
%reload_ext autoreload
print(MAX_FEATURES, ' features', N_TREES,'trees; ',
      LEARN_RATE,'learn_rate; ', MAX_DEPTH, 'max_dpth; ',
      MIN_IN_LEAF, 'min_in_leaf')
classifier_report(gb_pipe, Xa_test, ya_test,
                  'Gradient Boosting Classifier on test set A:')

2000  features 300 trees;  0.1 learn_rate;  15 max_dpth;  2 min_in_leaf


AttributeError: lower not found

## Split comments into separate sentences

In [ ]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

In [ ]:
small.columns

In [ ]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [ ]:
small.shape

In [ ]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(3)

## Sentence-level prep & cleaning

In [ ]:
%reload_ext autoreload
from utils import split_n_lower, not_about_support

### Split into words and lower the case

In [ ]:
sentences['words'] = sentences['sentence'].apply(lambda s: split_n_lower(s))

In [ ]:
print(sentences.shape)
sentences.head(3)

### Keep support-related sentences as they probably have impact on rating

In [ ]:
# on_movies_filter = [not_about_support(word) for word in sentences['words']]
sentences_on_movie = sentences #[on_movies_filter]

print('Removing {} records'.format(sentences.shape[0]- sentences_on_movie.shape[0]))

In [ ]:
sentences_on_movie.shape

### Base case: A reviews with objective and subjective sentences

In [ ]:
# Kernel dies here at 50K samples
all_reviews_groups = sentences_on_movie.groupby(['reviewerID','asin'])


In [ ]:
all_reviews_stars = all_reviews_groups['overall'].mean()
all_reviews_stars[:3]

In [ ]:
all_reviews_comments = all_reviews_groups['words'].sum()
print(sentences_on_movie.iloc[0, 4])
print(all_reviews_comments.shape)
print(all_reviews_comments[0])
len(all_reviews_comments)

### Remove objective sentences for case B using obj-subj model

In [ ]:
# pd.set_option('display.max_colwidth', -1)
# sentences_on_movie['sentence']
sentences_on_movie.shape

In [ ]:
sentences_on_movie.head(2)

#### Vectorize along the word space of the obj-subj training set

In [ ]:
tfidf = pickle.load(open('pickles/Obj-Subj_tfidf.pkl', 'rb'))
len(tfidf.vocabulary_)

In [ ]:
sentences_tfidf = tfidf.transform(sentences_on_movie['sentence']).todense()

In [ ]:
sentences_tfidf.shape

#### Apply the obj-subj model

In [ ]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open('pickles/GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'rb')
gb_model = pickle.load(pickle_in)

In [ ]:
y_test = gb_model.predict(sentences_tfidf)
len(y_test)

In [ ]:
subjective_sentences = sentences_on_movie[y_test == 1]

In [ ]:
display(Markdown('### Removing {} objective sentences'
                 .format(len(y_test) - len(subjective_sentences))))

In [ ]:
subjective_sentences.head(2)

#### Merge the sentences back into paragraph reviews

In [ ]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])
subj_reviews_stars = subj_groups['overall'].mean()
# subjective_reviewssubjective_reviews['sentence'].apply(lambda x: x.sum())
# subjective_reviews_reviews = 
subj_reviews_stars[:3]

In [ ]:
subj_review_comments = subj_groups['words'].sum()
print(subj_review_comments.shape)
print(subj_review_comments[0])
subj_review_comments[:3]

### Check that stars still correspond to the right movie

In [ ]:
start = 6000
end = 6010
all_reviews_comments.loc[('A33Z7JTV7SSW9Y', '0718000315')]


In [ ]:
print(all_reviews_stars.loc[('A33Z7JTV7SSW9Y', '0718000315')])
print(sentences_on_movie.loc[sentences_on_movie['reviewerID']=='A33Z7JTV7SSW9Y']) 
# and sentences_on_movie['asin']=='0718000315'])

In [ ]:
pd.options.display.max_colwidth = -1
print(small.loc[small['reviewerID']=='A33Z7JTV7SSW9Y'])

In [ ]:
sentences_on_movie[start:end]

## Create emotion vectors

In [ ]:
print('Total number of reviews:', all_reviews_comments.shape[0])
print('Total number of subjective reviews:', subj_review_comments.shape[0])

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
all_reviews_emotions = emote.vectorize(all_reviews_comments)
print(all_reviews_emotions.shape)

In [ ]:
# emote.emotions_in_text

In [ ]:
all_reviews_emotions.shape

In [ ]:
# all_revs_with_emotions = all_reviews_emotions[emote.emotions_in_text == True]

In [ ]:
# print(all_revs_with_emotions.shape)
# all_revs_stars = all_reviews_stars[emote.emotions_in_text]
all_reviews_emotions[0]

In [ ]:
subj_reviews_emotions = emote.vectorize(subj_review_comments)
print(subj_reviews_emotions.shape)
subj_reviews_emotions[0]

## Fit a model on base case (all comments) for star rating prediction

In [ ]:
X_subj_train, X_subj_cv, y_subj_train, y_subj_cv = train_test_split(
    subj_reviews_emotions, subj_reviews_stars, test_size=0.2, random_state=0)
X_subj_train.shape

In [ ]:
gbc_subj = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbc_subj.fit(X_subj_train, y_subj_train)

In [ ]:
print('Gradient Boosting Classifier')
print('Training score using all comments: {0:.2f}'
      .format(gbc_all.score(X_train, y_train)))
print('CV score using all comments: {0:.2f}'
      .format(gbc_all.score(X_cv, y_cv)))
print('')

# print('Training score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_train, y_subj_train)))
# print('CV score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_cv, y_subj_cv)))


## Other techniques

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(random_state=0, solver='lbfgs',
#                        multi_class='multinomial',max_iter=1000)
# lr.fit(X_subj_train, y_subj_train)
# print(lr.score(X_subj_train, y_subj_train))
# print(lr.score(X_subj_cv, y_subj_cv))

In [ ]:
ols_all = sm.OLS(y_train, X_train)
results_all = ols_all.fit()
results_all.summary()

In [ ]:
ols_subj = sm.OLS(y_subj_train, X_subj_train)
results_subj = ols_subj.fit()
results_subj.summary()

In [ ]:
import seaborn as sns

all_reviews_emotions, all_reviews_stars

sns.heatmap(raw_df.corr(), annot=True)